In [1]:
import pandas as pd
import json
import requests
import climateworks_datamosh as cw

In [100]:
%load_ext autoreload
%autoreload 2

In [108]:
from importlib import reload

In [109]:
reload(cw)

<module 'climateworks_datamosh' from '/mnt/labbook/code/climateworks_datamosh.py'>

# get (almost) all the data
### would need to retrieve again by iso3 code to make sure you really get all the records

In [45]:
payload={"start_year": 2017, "end_year": 2018, "regions": ["CHN"]}
datatable, error_pages = cw.get_cw_data(payload)

[]

In [85]:
allghg = datatable.loc[datatable["gas"] == "All GHG"].copy()
allghg = allghg.drop_duplicates(["country", "sector", "size"])

In [80]:
# allghg.to_excel("../output/allghg 2017.xlsx", encoding="utf8")

# get top 9 emitters

In [128]:
allghg = pd.read_excel("../output/allghg 2017.xlsx", encoding="utf8")

In [62]:
allghg.loc[(allghg["iso_code3"] == "CHN") & (allghg["sector"] == "Total excluding LUCF")]

,id,iso_code3,country,data_source,sector,gas,unit,emissions,size
7,541650,CHN,China,CAIT,Total excluding LUCF,All GHG,MtCO₂e,"[{'year': 2017, 'value': 12090.97}]",12090.97


In [129]:
allghg.loc[(allghg["country"] == "World") & (allghg["sector"] == "Total excluding LUCF")]

,Unnamed: 0,id,iso_code3,country,data_source,sector,gas,unit,emissions,size
2551,3,548970,WORLD,World,CAIT,Total excluding LUCF,All GHG,MtCO₂e,"[{'year': 2017, 'value': 46730.35}]",46730.35


In [52]:
allghg_c = allghg.loc[~allghg["country"].isin(["World", "European Union (27)"])].copy().sort_values(by=["size"], ascending=False)

allghg_c.loc[(allghg_c["sector"] == "Total excluding LUCF")]["size"].sum()

44122.76

In [58]:
top9_names = allghg_c.loc[allghg_c["sector"] == "Total excluding LUCF"].head(9)[["iso_code3", "country"]]

In [94]:
top9_names.to_excel("../output/top9_names2017.xlsx")

# format data for chart

In [90]:
top9data = top9_names["iso_code3"].apply(
    lambda c: cw.get_cw_data({"start_year": 2017, "end_year": 2018, "regions": [c]}))

3
3
3
3
3
3
3
3
3


In [121]:
ghg_fs = []
dfs = []
for d in top9data:
    allghg = d[0].loc[d[0]["gas"] == "All GHG"].copy()
    dfs.append(d[0])
    ghg_fs.append(cw.format_sectors(allghg))

China
12090.97
10037.669999999998
United States
5840.71
5100.78
India
3230.2700000000004
2307.8199999999997
Russia
2496.2999999999997
2227.21
Japan
1235.31
1135.3600000000001
Brazil
1063.59
459.14
Indonesia
912.33
548.52
Germany
832.67
736.5200000000001
Iran
814.7399999999999
704.8700000000001


In [132]:
top9ghg = pd.concat(dfs)
top9ghg = top9ghg.loc[top9ghg["gas"] == "All GHG"].copy()

In [115]:
ghg_fs[0]

{'name': 'China',
 'size': 12090.97,
 'children': [{'name': 'energy',
   'size': 10037.65,
   'children': [{'name': 'electricity', 'size': 4904.12},
    {'name': 'manufacturing', 'size': 2734.18},
    {'name': 'transportation', 'size': 880.9},
    {'name': 'building', 'size': 533.08},
    {'name': 'other fuels', 'size': 290.6},
    {'name': 'fugitive', 'size': 694.79}]},
  {'name': 'industrial processes', 'size': 1144.49},
  {'name': 'agriculture', 'size': 717.22},
  {'name': 'waste', 'size': 191.61}]}

# validate World numbers

In [135]:
worldghg = allghg.loc[(allghg["country"] == "World")]

In [137]:
names_map = {**cw.sector_names, **cw.e_subsectors}
worldghg.loc[:, "name"] = worldghg["sector"].apply(lambda s: names_map.get(s, s))

In [139]:
worldghg.loc[worldghg["name"] == "Total excluding LUCF", "size"].values[0]

46730.35

In [141]:
worldghg.loc[worldghg["name"].isin(list(sector_names.values()))]["size"].sum()

46730.35

In [144]:
worldghg.loc[worldghg["name"]=="energy"]["size"]

2553    36435.64
Name: size, dtype: float64

In [147]:
worldghg.loc[worldghg["name"].isin(list(cw.e_subsectors.values()))]["size"].sum()

36435.65

In [152]:
orecord = cw.other_record(allghg.loc[(allghg["country"] == "World")], top9ghg)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [153]:
orecord

{'name': 'Other',
 'size': 18213.47,
 'children': [{'name': 'energy',
   'size': 13177.77,
   'children': [{'name': 'electricity', 'size': 4770.11},
    {'name': 'manufacturing', 'size': 1610.98},
    {'name': 'transportation', 'size': 4089.15},
    {'name': 'building', 'size': 1055.8},
    {'name': 'other fuels', 'size': 775.86},
    {'name': 'fugitive', 'size': 875.86}]},
  {'name': 'industrial processes', 'size': 1073.73},
  {'name': 'agriculture', 'size': 3163.61},
  {'name': 'waste', 'size': 798.35}]}

In [154]:
ghg2017 = {"name": "global ghg sources", "children": [orecord]+ghg_fs}

In [155]:
with open("../output/ghg2017_source_countries.json", 'w') as f:
    json.dump(ghg2017, f, indent=2)